In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 30    # steepness factor
num_blocks = 2    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_exact_m.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:39:54<00:00,  9.59s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      2.594804               0.0      9.593500
std       3.358934               0.0     10.182139
min       0.000433               0.0      0.341703
25%       0.171067               0.0      3.415910
50%       1.217010               0.0      6.312147
75%       3.839457               0.0     11.383512
max      20.375223               0.0     61.098220
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible。。。。。。 solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rnd_m.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:36:18<00:00,  9.38s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      7.842982          0.109557      9.373762
std       4.567989          0.599686     16.713101
min       0.058280          0.000000      0.670847
25%       4.527683          0.000000      0.905526
50%       7.221738          0.000000      1.127736
75%      10.252552          0.000000      8.059175
max      27.025234          9.499012     60.447122
Number of infeasible solution: 96


## Heuristic - RENS

In [10]:
mmfrom src.heuristic import rens

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    #
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    try:
        # get optimal rounding
        xval, objval = rens(xval_rel, model)
    except:
        # no feasible rounding
        xval, objval = None, None
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    if xval:
        # get optimal rounding
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    else:
        sols.append(None)
        objvals.append(None)
        conviols.append(np.inf)
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rens_m.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [3:08:30<00:00, 11.31s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      4.990809          0.047468     11.306315
std      40.216456          0.406606     44.503785
min       0.000433          0.000000      1.286747
25%       0.201606          0.000000      2.004303
50%       1.280895          0.000000      2.508087
75%       4.101634          0.000000      8.650895
max    1149.403532          5.094683   1325.599293
Number of infeasible solution: 20


## Heuristic - N1

In [12]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_m.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:02<00:00,  8.17it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean    532.157325               0.0      0.121405
std     363.175683               0.0      0.030642
min      75.030866               0.0      0.102742
25%     186.276981               0.0      0.107533
50%     470.379688               0.0      0.108800
75%     889.177330               0.0      0.122691
max    1217.174467               0.0      0.263184
Number of infeasible solution: 0


## Heuristic - N3

In [14]:
model_heur = model.first_solution_heuristic(nodes_limit=3)

In [15]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n3_m.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:03<00:00,  2.36it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean    124.889115               0.0      0.422636
std     238.139014               0.0      2.333885
min       0.037609               0.0      0.103508
25%      10.780444               0.0      0.110267
50%      29.830091               0.0      0.153013
75%      78.745917               0.0      0.173014
max     972.548668               0.0     51.869199
Number of infeasible solution: 0


## Learnable Rounding

In [16]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [17]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# define Rosenbrock objective functions and constraints for both problem types
obj_rel, constrs_rel = nmRosenbrock(["x"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
obj_rnd, constrs_rnd = nmRosenbrock(["x_rnd"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = [smap, rnd]
loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
problem = nm.problem.Problem(components, loss)

In [18]:
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(problem.parameters(), lr=lr)
# create a trainer for the problem
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test, optimizer, 
                            epochs=epochs, patience=patience, warmup=warmup)
# training for the rounding problem
best_model = trainer.train()
# load best model dict
problem.load_state_dict(best_model)

epoch: 0  train_loss: 141.621337890625
epoch: 1  train_loss: 54.17548370361328
epoch: 2  train_loss: 36.103633880615234
epoch: 3  train_loss: 29.21131134033203
epoch: 4  train_loss: 25.601943969726562
epoch: 5  train_loss: 23.98379898071289
epoch: 6  train_loss: 21.947715759277344
epoch: 7  train_loss: 20.99851417541504
epoch: 8  train_loss: 19.32303237915039
epoch: 9  train_loss: 18.250574111938477
epoch: 10  train_loss: 16.494781494140625
epoch: 11  train_loss: 16.3751277923584
epoch: 12  train_loss: 15.23391056060791
epoch: 13  train_loss: 15.205641746520996
epoch: 14  train_loss: 14.716923713684082
epoch: 15  train_loss: 14.3939208984375
epoch: 16  train_loss: 15.067134857177734
epoch: 17  train_loss: 13.788334846496582
epoch: 18  train_loss: 13.365486145019531
epoch: 19  train_loss: 13.073370933532715
epoch: 20  train_loss: 13.448643684387207
epoch: 21  train_loss: 13.225969314575195
epoch: 22  train_loss: 12.926955223083496
epoch: 23  train_loss: 13.177831649780273
epoch: 24  tra

<All keys matched successfully>

In [19]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = output["test_x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_nm_m.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 386.35it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      9.602252          0.000048      0.002232
std       6.035786          0.001523      0.000713
min       0.118130          0.000000      0.000997
25%       4.764137          0.000000      0.002000
50%       9.106999          0.000000      0.002002
75%      12.689323          0.000000      0.002511
max      35.417544          0.048146      0.011335
Number of infeasible solution: 1
